In [1]:
import tensorflow as tf
import numpy as np
from network import Network
from pinn import Pinn
from optimizer import L_BFGS_B

2022-12-31 20:19:45.144780: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-31 20:19:45.258769: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-12-31 20:19:45.258794: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-31 20:19:45.999841: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file

In [2]:
N_train = 3000
N_test = 1000

t = tf.random.uniform((N_train, 1), 0, 1)
x = tf.random.uniform((N_train, 1), -1, 1)
tx = tf.concat([t, x], axis=1)

t0 = tf.zeros((N_train, 1))
x0 = tf.random.uniform((N_train, 1), -1, 1)
tx0 = tf.concat([t0, x0], axis=1)

tb = tf.random.uniform((N_train, 1), 0, 4)
xb = -1 + 2*tf.keras.backend.random_bernoulli((N_train, 1), p=0.5)
txb = tf.concat([tb, xb], axis=1)

inputs = [tx, tx0, txb]

2022-12-31 20:19:56.783402: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-12-31 20:19:56.783432: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-31 20:19:56.783452: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (noel-IdeaPad-3-15ALC6): /proc/driver/nvidia/version does not exist
2022-12-31 20:19:56.783774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def u0fun(tx, a=np.pi):
    t, x = tx[..., 0, None], tx[..., 1, None]
    return np.sin(a*x)

In [4]:
res = np.zeros((N_train, 1))
u0 = u0fun(tx0)
# u_t0 = np.zeros((N_train, 1))  # Not necessary, in theory
u_b = np.zeros((N_train, 1))

outputs = [res, res, u0, u_b]

In [5]:
model = Network.build(hid_neurons=[16])
pinn = Pinn(model, c=1.0).pinn
solver = L_BFGS_B(model=pinn, x_train=inputs, y_train=outputs, maxiter=80, factr=2.5e3)

In [6]:
pinn.compile(optimizer='adam', loss='mse')

In [7]:
pinn.fit(x=solver.x_train, y=solver.y_train, batch_size=32, epochs=100)

Epoch 1/100
94/94 [==============================] - 6s 2ms/step - loss: 5.3328 - tf.__operators__.add_loss: 2.4903 - tf.math.subtract_loss: 0.2793 - gradient_layer_loss: 0.5417 - gradient_layer_1_loss: 2.0213
Epoch 2/100
94/94 [==============================] - 0s 3ms/step - loss: 1.3200 - tf.__operators__.add_loss: 0.4258 - tf.math.subtract_loss: 0.1638 - gradient_layer_loss: 0.4504 - gradient_layer_1_loss: 0.2800
Epoch 3/100
94/94 [==============================] - 0s 3ms/step - loss: 0.6226 - tf.__operators__.add_loss: 0.0945 - tf.math.subtract_loss: 0.1021 - gradient_layer_loss: 0.3699 - gradient_layer_1_loss: 0.0561
Epoch 4/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4666 - tf.__operators__.add_loss: 0.0401 - tf.math.subtract_loss: 0.0690 - gradient_layer_loss: 0.3131 - gradient_layer_1_loss: 0.0445
Epoch 5/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4041 - tf.__operators__.add_loss: 0.0259 - tf.math.subtract_loss: 0.0534 - gradient_l